In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan, normal_ad
from sklearn.preprocessing import OneHotEncoder
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import statsmodels.api as sm



In [ ]:
df = pd.read_csv('cars_clean_clean_clean.xls')

In [ ]:
final_featues = ['log_age',
               'brand_Geely',
               'is_restyling',
               'color_group_Холодные',
               'log_power_2',
               'city_group_Москва',
               'gearbox_механика',
               'drive_полный',
               'car_class_Crossover']
numeric_feats = ['mileage', 'power_1', 'power_2', 'age']
cat_feats = ['brand', 'color_group', 'gearbox', 'drive', 'fuel_type', 'city_group', 'car_class']
dummies = ['is_restyling', 'is_pro', 'is_max', 'is_premium']
target = 'price'



In [ ]:
def prepare_data(df, target_col, categorical_cols, numeric_cols=None, dummies=None):
    """Подготовка данных с обработкой категориальных и числовых признаков"""
    # Создание dummy-переменных
    encoder = OneHotEncoder(drop='first', sparse_output=False)
    encoded = encoder.fit_transform(df[categorical_cols])
    encoded_df = pd.DataFrame(encoded,
                            columns=encoder.get_feature_names_out(categorical_cols))

    # Обработка числовых признаков
    if numeric_cols is None:
        numeric_cols = df.drop(columns=[target_col] + categorical_cols).columns

    X_numeric = df[numeric_cols].copy()
    X_dum = df[dummies].copy()

    # Создание преобразованных признаков
    X_transformed = pd.DataFrame()
    for col in numeric_cols:
        X_transformed[f'log_{col}'] = np.log1p(X_numeric[col])
        X_transformed[f'sq_{col}'] = X_numeric[col]**2

    # Объединение всех признаков
    X = pd.concat([X_numeric, X_transformed, encoded_df, X_dum], axis=1)
    y = df[target_col]

    return X, y, np.log1p(y), encoder

In [ ]:
# Подготовка данных
X_full, y, log_y, encoder = prepare_data(df, target, cat_feats, numeric_feats, dummies)

In [ ]:
X_final = sm.add_constant(X_full[final_featues])
model_linear_final = sm.OLS(log_y, X_final).fit(cov_type='HC3')
print(model_linear_final.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.894
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     1216.
Date:                Thu, 08 May 2025   Prob (F-statistic):               0.00
Time:                        15:18:30   Log-Likelihood:                 355.17
No. Observations:                1315   AIC:                            -690.3
Df Residuals:                    1305   BIC:                            -638.5
Df Model:                           9                                         
Covariance Type:                  HC3                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   11.3904 

# Новое наблюдение

In [ ]:
model_linear_final.predict(X_final)

,0
0,14.666750
1,13.567497
2,14.666750
3,14.673421
4,14.279631
...,...
1310,15.174606
1311,14.762645
1312,14.680269
1313,14.492286


In [ ]:
data = {
    'const': [1],
    'log_age': [np.log(2)],
    'brand_Geely': [1],
    'is_restyling': [0],
    'color_group_Холодные': [1],
    'log_power_2': [np.log(361)],
    'city_group_Москва': [1],
    'gearbox_механика': [0],
    'drive_полный': [0],
    'car_class_Crossover': [0]
}
new_obs = pd.DataFrame(data)

pred = model_linear_final.predict(new_obs)

# Рассчитываем доверительный интервал
_, pred_lower, pred_upper = wls_prediction_std(model_linear_final, exog=new_obs, alpha=0.05)

print(f"Предсказанная цена (в логарифмах): {pred[0]:.4f}")
print(f"Доверительный интервал 95%: [{pred_lower[0]:.4f}, {pred_upper[0]:.4f}]")

# Преобразуем обратно из логарифмов
print(f"\nПредсказанная цена (в исходной шкале): {np.exp(pred[0]):.2f} RUB")
print(f"Доверительный интервал 95%: [{np.exp(pred_lower[0]):.2f}, {np.exp(pred_upper[0]):.2f}] RUB")

Предсказанная цена (в логарифмах): 15.2159
Доверительный интервал 95%: [14.8432, 15.5886]

Предсказанная цена (в исходной шкале): 4056637.95 RUB
Доверительный интервал 95%: [2794539.51, 5888738.17] RUB


In [ ]:
new_obs

,const,log_age,brand_Geely,is_restyling,color_group_Холодные,log_power_2,city_group_Москва,gearbox_механика,drive_полный,car_class_Crossover
0,1,0.693147,1,0,1,5.888878,1,0,0,0


In [ ]:
X_full[final_featues].shape

(1315, 9)

In [ ]:
X_full

,mileage,power_1,power_2,age,log_mileage,sq_mileage,log_power_1,sq_power_1,log_power_2,sq_power_2,...,city_group_Прочие города,car_class_Crossover,car_class_Luxury SUV,car_class_Other,car_class_SUV,car_class_Sedan,is_restyling,is_pro,is_max,is_premium
0,12000.0,1.5,177.0,1.0,9.392745,1.440000e+08,0.916291,2.25,5.181784,31329.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,0
1,52460.0,1.5,113.0,1.0,10.867825,2.752052e+09,0.916291,2.25,4.736198,12769.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0,1,0,0
2,6308.0,1.5,177.0,1.0,8.749732,3.979086e+07,0.916291,2.25,5.181784,31329.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,0
3,17169.0,2.0,200.0,2.0,9.750919,2.947746e+08,1.098612,4.00,5.303305,40000.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0,0
4,22000.0,1.5,147.0,3.0,9.998843,4.840000e+08,0.916291,2.25,4.997212,21609.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,10.0,2.0,238.0,0.0,2.397895,1.000000e+02,1.098612,4.00,5.476464,56644.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0,1
1311,1.0,1.5,181.0,1.0,0.693147,1.000000e+00,0.916291,2.25,5.204007,32761.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0
1312,10.0,1.5,181.0,1.0,2.397895,1.000000e+02,0.916291,2.25,5.204007,32761.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0
1313,58000.0,1.5,177.0,3.0,10.968216,3.364000e+09,0.916291,2.25,5.181784,31329.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0,1,0,0


In [ ]:
sm.add_constant(new_obs)

,const,log_age,brand_Geely,is_restyling,color_group_Холодные,log_power_2,city_group_Москва,gearbox_механика,drive_полный,car_class_Crossover
0,1,0.693147,1,0,1,5.888878,1,0,0,0


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_reset
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.compat import lzip
from scipy import stats
from statsmodels.stats.diagnostic import het_breuschpagan, normal_ad, linear_rainbow
from statsmodels.stats.outliers_influence import reset_ramsey
from statsmodels.base.model import LikelihoodModelResults
from statsmodels.tools.eval_measures import mse
from statsmodels.stats.api import anova_lm
from statsmodels.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import linear_reset, linear_rainbow, het_breuschpagan
from statsmodels.stats.stattools import jarque_bera
from scipy.stats import boxcox_normmax
import numpy as np
from statsmodels.stats.diagnostic import linear_reset, linear_rainbow
from statsmodels.stats.stattools import jarque_bera
from scipy.stats import boxcox_normmax
from statsmodels.stats.diagnostic import linear_reset, linear_rainbow
from scipy.stats import jarque_bera, boxcox_normmax


Протестируем спецификацию текущей модели:

In [ ]:
# Ramsey RESET test
reset_test = linear_reset(model_linear_final, power=2, use_f=True)
reset_result = {
    'F-statistic': reset_test.fvalue,
    'p-value': reset_test.pvalue
}

# Проверка нормальности остатков
jb_stat, jb_pvalue, _, _ = sm.stats.jarque_bera(model_linear_final.resid)

# Box-Cox тест требует положительного таргета и модели в исходной шкале
y_original = np.expm1(model_linear_final.model.endog)
X_bc = model_linear_final.model.exog
boxcox_result = stats.boxcox_normmax(y_original, method='mle')

# PE-тест: Rainbow test
rainbow_stat, rainbow_p = linear_rainbow(model_linear_final)

results = {
    "Ramsey RESET Test (misspecification)": reset_result,
    "Jarque-Bera Test (normality)": {"JB-statistic": jb_stat, "p-value": jb_pvalue},
    "Box-Cox lambda (functional form)": boxcox_result,
    "Rainbow Test (PE-test for functional form)": {"statistic": rainbow_stat, "p-value": rainbow_p}
}

results


{'Ramsey RESET Test (misspecification)': {'F-statistic': 152.8273592856319,
  'p-value': np.float64(2.8180231479666e-33)},
 'Jarque-Bera Test (normality)': {'JB-statistic': np.float64(40.686315445939044),
  'p-value': np.float64(1.4624426913828906e-09)},
 'Box-Cox lambda (functional form)': np.float64(0.9095779674695604),
 'Rainbow Test (PE-test for functional form)': {'statistic': np.float64(1.8594433191125304),
  'p-value': np.float64(2.0823466169694155e-15)}}

1. Тест Рамсея RESET — на пропущенные переменные:

F-статистика: 152.83 \\

p-value: < 0.0001 \\

Нулевая гипотеза: модель правильно специфицирована. \\

Вывод: гипотеза отвергается, в модели, вероятно, пропущены важные переменные или взаимодействия.

2. Тест Жарка-Бера (нормальность остатков): \\
JB-статистика: 40.69 \\
p-value: < 0.0001 \\
Остатки не нормально распределены, что может сказаться на корректности t- и F-статистик, особенно в малой выборке.

3. Тест Бокса-Кокса (λ = 0.91): \\
Оптимальное преобразование близко к логарифмическому (λ ≈ 0.91). Это подтверждает логарифмирование зависимой переменной как удачный выбор.

4. Rainbow Test (PE-тест на функциональную форму): \\
Статистика: 1.86 \\
p-value: < 0.0001 \\
Нулевая гипотеза: модель корректно специфицирована.
Вывод: гипотеза отвергается. Есть признаки неправильной функциональной формы — возможно, не хватает взаимодействий, квадратичных или логарифмических переменных.

### Изменим модель

На основе содержательного смысла и предыдущего анализа:

log_age × log_power_2 — старые машины с высокой мощностью могут дешеветь быстрее \\
drive_полный × car_class_Crossover — полный привод имеет смысл в кроссоверах \\
brand_Geely × is_restyling — премия за рестайлинг может зависеть от бренда \\
color_group_Холодные × car_class_Crossover — определённые цвета популярнее у кроссоверов

In [ ]:
# Создаём перекрёстные переменные
X_full['int_age_power'] = X_full['log_age'] * X_full['log_power_2']
X_full['int_drive_crossover'] = X_full['drive_полный'] * X_full['car_class_Crossover']
X_full['int_brand_restyling'] = X_full['brand_Geely'] * X_full['is_restyling']
X_full['int_color_crossover'] = X_full['color_group_Холодные'] * X_full['car_class_Crossover']

# Обновим список признаков
final_features_interact = final_featues + [
    'int_age_power',
    'int_drive_crossover',
    'int_brand_restyling',
    'int_color_crossover'
]

# Построим новую модель
X_inter = sm.add_constant(X_full[final_features_interact])
model_linear_inter = sm.OLS(log_y, X_inter).fit(cov_type='HC3')
print(model_linear_inter.summary())


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.911
Method:                 Least Squares   F-statistic:                     1192.
Date:                Thu, 08 May 2025   Prob (F-statistic):               0.00
Time:                        15:19:04   Log-Likelihood:                 480.51
No. Observations:                1315   AIC:                            -933.0
Df Residuals:                    1301   BIC:                            -860.5
Df Model:                          13                                         
Covariance Type:                  HC3                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   15.5147 

In [18]:
# Проверка спецификации модели
reset_test = linear_reset(model_linear_inter, power=2, use_f=True)
rainbow_stat, rainbow_p = linear_rainbow(model_linear_inter)
jb_stat, jb_p, skew, kurt = jarque_bera(model_linear_inter.resid)
lambda_bc = boxcox_normmax(y)  # y в исходной шкале, не логарифм

# Сбор результатов
specification_results = {
    'Ramsey RESET Test (пропуск переменных)': {
        'F-statistic': reset_test.fvalue,
        'p-value': reset_test.pvalue
    },
    'Rainbow Test (функц. форма, PE test)': {
        'statistic': rainbow_stat,
        'p-value': rainbow_p
    },
    'Jarque-Bera Test (нормальность)': {
        'JB-statistic': jb_stat,
        'p-value': jb_p,
        'skew': skew,
        'kurtosis': kurt
    },
    'Box-Cox lambda (оптимальная трансформация)': lambda_bc
}

# Вывод
for test, result in specification_results.items():
    print(f"\n{test}:")
    if isinstance(result, dict):
        for key, value in result.items():
            print(f"  {key}: {value:.5f}")
    else:
        print(f"  λ = {result:.5f}")


ValueError: not enough values to unpack (expected 4, got 2)

1. Ramsey RESET Test (пропущенные переменные) \\
F-statistic = 40.01, p-value = 0.00000 \\
Интерпретация: \\
Тест отвергает нулевую гипотезу о правильной спецификации модели. Это значит, что в модели, скорее всего, отсутствуют важные переменные или связи между переменными (например, взаимодействия или полиномы).

2. Rainbow Test (ошибочная функциональная форма) \\
Statistic = 2.11, p-value = 0.00000 \\
Интерпретация: \\
Также отвергается гипотеза о корректной линейной форме модели. Возможно, связь между некоторыми признаками и логарифмом цены нелинейна.

3. Jarque-Bera Test (нормальность остатков) \\
JB = 73.11, p = 0.00000, skew ≈ 0.043, kurtosis ≈ 4.15 \\
Интерпретация: \\
Остатки не распределены нормально, что может нарушать предпосылки OLS и влиять на корректность доверительных интервалов.

4. Box-Cox λ = 0.93889 \\
Значение λ близко к 1 → текущая логарифмическая трансформация цены (log_y) адекватна \\
λ ≠ 1 → возможно, небольшое улучшение даст Box-Cox-преобразование вместо логарифма

### Изменим модель, добавим новые признаки, кросс-переменные, квадраты, логарифмы


In [ ]:
final_features = [
    'log_age',
    'brand_Geely',
    'is_restyling',
    'color_group_Холодные',
    'log_power_2',
    'city_group_Москва',
    'gearbox_механика',
    'drive_полный',
    'car_class_Crossover'
]


In [ ]:
X_full['log_age_x_log_power_2'] = X_full['log_age'] * X_full['log_power_2']
X_full['gearbox_механика_x_drive_полный'] = X_full['gearbox_механика'] * X_full['drive_полный']
X_full['brand_Geely_x_city_Москва'] = X_full['brand_Geely'] * X_full['city_group_Москва']


In [ ]:
X_full['log_power_2_sq'] = X_full['log_power_2'] ** 2
X_full['log_age_sq'] = X_full['log_age'] ** 2


In [ ]:
improved_features = final_features + [
    'log_age_x_log_power_2',
    'gearbox_механика_x_drive_полный',
    'brand_Geely_x_city_Москва',
    'log_power_2_sq',
    'log_age_sq'
]


In [ ]:
import statsmodels.api as sm

X_improved = sm.add_constant(X_full[improved_features])
model_improved = sm.OLS(log_y, X_improved).fit(cov_type='HC3')
print(model_improved.summary())


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.911
Method:                 Least Squares   F-statistic:                     1102.
Date:                Thu, 08 May 2025   Prob (F-statistic):               0.00
Time:                        15:19:32   Log-Likelihood:                 481.63
No. Observations:                1315   AIC:                            -933.3
Df Residuals:                    1300   BIC:                            -855.5
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [ ]:
from scipy.stats import jarque_bera, skew, kurtosis
from statsmodels.stats.diagnostic import linear_reset, linear_rainbow
from scipy.stats import boxcox_normmax

# Ramsey RESET
reset_test = linear_reset(model_improved, power=2, use_f=True)

# Rainbow test (functional form)
rainbow_stat, rainbow_p = linear_rainbow(model_improved)

# Jarque-Bera + распределение остатков
jb_stat, jb_p = jarque_bera(model_improved.resid)
skw = skew(model_improved.resid)
krt = kurtosis(model_improved.resid, fisher=False)

# Box-Cox (на оригинальной шкале y)
lambda_bc = boxcox_normmax(y)

# Свод результатов
spec_results = {
    'Ramsey RESET Test (misspecification)': {
        'F-statistic': reset_test.fvalue,
        'p-value': reset_test.pvalue
    },
    'Rainbow Test (functional form)': {
        'statistic': rainbow_stat,
        'p-value': rainbow_p
    },
    'Jarque-Bera Test (normality)': {
        'JB-statistic': jb_stat,
        'p-value': jb_p,
        'skew': skw,
        'kurtosis': krt
    },
    'Box-Cox lambda (functional form)': lambda_bc
}

import pprint
pprint.pprint(spec_results, sort_dicts=False)


{'Ramsey RESET Test (misspecification)': {'F-statistic': 91.74003411003926,
                                          'p-value': np.float64(4.80491370824291e-21)},
 'Rainbow Test (functional form)': {'statistic': np.float64(1.8470836063683809),
                                    'p-value': np.float64(4.516512076510267e-15)},
 'Jarque-Bera Test (normality)': {'JB-statistic': np.float64(24.752340977515246),
                                  'p-value': np.float64(4.217911368730192e-06),
                                  'skew': np.float64(0.06745094735351762),
                                  'kurtosis': np.float64(3.658449110019399)},
 'Box-Cox lambda (functional form)': np.float64(0.9388924080164085)}


1. Ramsey RESET Test (ошибка спецификации) \\
F = 91.74, p < 0.001 → гипотеза о корректной спецификации отвергается \\
Это означает пропущены важные переменные, взаимодействия или неверная функциональная форма \\


2. Rainbow Test (PE-test на функциональную форму) \\
stat = 1.85, p < 0.001 → также говорит о неадекватной функциональной форме \\


3. Jarque-Bera Test (нормальность остатков) \\
JB = 24.75, p < 0.001 → остатки не нормальны, но: \\
Skew = 0.067 — практически симметрия \\
Kurtosis = 3.66 — немного выше нормы (3) \\

4. Box-Cox λ = 0.94 \\
λ ≈ 1 → логарифмирование (log_y) почти оптимально \\
Значит, модель с log_y как таргетом — обоснована \\

### Изменим модель, добавим новые признаки

In [ ]:
# Перекрёстные и квадратичные переменные
X_full['log_age_x_log_power_2'] = X_full['log_age'] * X_full['log_power_2']
X_full['log_power_2_sq'] = X_full['log_power_2'] ** 2
X_full['drive_x_geely'] = X_full['drive_полный'] * X_full['brand_Geely']
X_full['gearbox_mech_x_power'] = X_full['gearbox_механика'] * X_full['log_power_2']


In [ ]:
final_features_extended = [
    'log_age',
    'brand_Geely',
    'is_restyling',
    'color_group_Холодные',
    'log_power_2',
    'city_group_Москва',
    'gearbox_механика',
    'drive_полный',
    'car_class_Crossover',
    'log_age_x_log_power_2',
    'log_power_2_sq',
    'drive_x_geely',
    'gearbox_mech_x_power'
]


In [ ]:
X_ext = sm.add_constant(X_full[final_features_extended])
model_improved = sm.OLS(log_y, X_ext).fit(cov_type='HC3')
print(model_improved.summary())


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.915
Method:                 Least Squares   F-statistic:                     1324.
Date:                Thu, 08 May 2025   Prob (F-statistic):               0.00
Time:                        15:33:06   Log-Likelihood:                 512.75
No. Observations:                1315   AIC:                            -997.5
Df Residuals:                    1301   BIC:                            -925.0
Df Model:                          13                                         
Covariance Type:                  HC3                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     7.29

In [ ]:
# Проверка спецификации улучшенной модели
reset_test = linear_reset(model_improved, power=2, use_f=True)
rainbow_stat, rainbow_p = linear_rainbow(model_improved)
jb_stat, jb_p = jarque_bera(model_improved.resid)
lambda_bc = boxcox_normmax(y)

# Сбор результатов
specification_results = {
    'Ramsey RESET Test (misspecification)': {
        'F-statistic': reset_test.fvalue,
        'p-value': reset_test.pvalue
    },
    'Rainbow Test (functional form)': {
        'statistic': rainbow_stat,
        'p-value': rainbow_p
    },
    'Jarque-Bera Test (normality)': {
        'JB-statistic': jb_stat,
        'p-value': jb_p
    },
    'Box-Cox lambda (functional form)': lambda_bc
}

import pprint
pprint.pprint(specification_results)

{'Box-Cox lambda (functional form)': np.float64(0.9388924080164085),
 'Jarque-Bera Test (normality)': {'JB-statistic': np.float64(133.58801002706727),
                                  'p-value': np.float64(9.81142748543865e-30)},
 'Rainbow Test (functional form)': {'p-value': np.float64(1.0406344959116691e-33),
                                    'statistic': np.float64(2.6230224904178185)},
 'Ramsey RESET Test (misspecification)': {'F-statistic': 0.2822088062642098,
                                          'p-value': np.float64(0.5953478781362649)}}


1. Ramsey RESET Test (пропуск переменных) \\
F-statistic = 0.2822, p-value = 0.5953 \\
Интерпретация: Нет оснований отвергнуть нулевую гипотезу — модель корректно специфицирована, существенных пропущенных переменных нет. \\

2. Rainbow Test (неправильная функциональная форма) \\
Statistic = 2.623, p-value ≈ 0.0000 \\
Интерпретация: Есть значимые отклонения от линейной функциональной формы на подвыборках. \\

3. Jarque-Bera Test (ненормальность остатков) \\
JB-statistic = 133.588, p-value ≈ 0.0000 \\
Интерпретация: Остатки не распределены нормально (возможно, из-за выбросов). \\

4. Box-Cox lambda = 0.9389 \\
Значение близко к 1 → подтверждает, что логарифмирование зависимой переменной (log(price)) — оправданно. \\


# Проверяем значимость и мультикол

In [ ]:
# убрала незначимые на уровне значимости 10%
final_features_extended = [
    'is_restyling',
    'color_group_Холодные',
    'city_group_Москва',
    'gearbox_механика',
    'log_age_x_log_power_2',
    'log_power_2_sq',
    'drive_x_geely',
    'gearbox_mech_x_power'
]

X_ext = sm.add_constant(X_full[final_features_extended])
model_improved = sm.OLS(log_y, X_ext).fit(cov_type='HC3')
print(model_improved.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.889
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                     1293.
Date:                Thu, 08 May 2025   Prob (F-statistic):               0.00
Time:                        15:35:45   Log-Likelihood:                 325.91
No. Observations:                1315   AIC:                            -633.8
Df Residuals:                    1306   BIC:                            -587.2
Df Model:                           8                                         
Covariance Type:                  HC3                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    12.89

In [ ]:
# Проверка спецификации улучшенной модели
reset_test = linear_reset(model_improved, power=2, use_f=True)
rainbow_stat, rainbow_p = linear_rainbow(model_improved)
jb_stat, jb_p = jarque_bera(model_improved.resid)
lambda_bc = boxcox_normmax(y)

# Сбор результатов
specification_results = {
    'Ramsey RESET Test (misspecification)': {
        'F-statistic': reset_test.fvalue,
        'p-value': reset_test.pvalue
    },
    'Rainbow Test (functional form)': {
        'statistic': rainbow_stat,
        'p-value': rainbow_p
    },
    'Jarque-Bera Test (normality)': {
        'JB-statistic': jb_stat,
        'p-value': jb_p
    },
    'Box-Cox lambda (functional form)': lambda_bc
}

import pprint
pprint.pprint(specification_results)


{'Box-Cox lambda (functional form)': np.float64(0.9388924080164085),
 'Jarque-Bera Test (normality)': {'JB-statistic': np.float64(69.37041844251074),
                                  'p-value': np.float64(8.63783753898133e-16)},
 'Rainbow Test (functional form)': {'p-value': np.float64(1.2785579455754882e-15),
                                    'statistic': np.float64(1.8680111502159769)},
 'Ramsey RESET Test (misspecification)': {'F-statistic': 340.73841419099693,
                                          'p-value': np.float64(8.814358593662435e-68)}}


In [ ]:
def check_assumptions(model, X, y):
    """Проверка предпосылок классической линейной регрессии"""
    # Нормальность остатков
    _, p_norm = normal_ad(model.resid)
    print(f"Тест на нормальность (p-value): {p_norm:.4f}")

    # Мультиколлинеарность
    vif = pd.DataFrame()
    vif['Variable'] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print("\nVIF факторов:")
    return vif.sort_values("VIF", ascending=False)

check_assumptions(model_improved, X_ext, log_y)

Тест на нормальность (p-value): 0.0000

VIF факторов:


,Variable,VIF
5,gearbox_механика,1814.318274
9,gearbox_mech_x_power,1726.150352
0,const,1368.731461
1,log_age,1234.854499
6,log_age_x_log_power_2,1103.156767
7,log_power_2_sq,9.859215
8,drive_x_geely,3.591299
2,is_restyling,1.669826
4,city_group_Москва,1.348351
3,color_group_Холодные,1.166094


In [ ]:
:
